In [1]:
import os
import pandas as pd
import altair as alt

/var/folders/kf/wy3gpbrx0tx0vwxf9qd1pzvm0000gn/T/ipykernel_17038/2612184196.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'altair'